In [4]:
%reset -fs
import numpy as np
import numpy.linalg as la
import scipy.sparse as sp
import matplotlib.pyplot as plt

# Uebungsblatt 3 - Aufgabe 1

## (a)

### LUP-Faktorisierung

In [5]:
def LUPFaktorisierung(A):
    if not A.shape[0] == A.shape[1]:
        print ("A is not square.")
        return 0,0
    N = A.shape[0]
    LU = np.copy(A.astype(np.float64))
    P = np.eye(N)

    npivot = 0

    #LU[0][0] unkritisch, da LU[0][0] = L[0][0] => (keine Division durch LU[1][1])

    for i in range(0,N):
        for j in range(0,N):
            if ( j < i ):
                # Berechnung von L
                LU[i,j] = str(((1/LU[j,j])*(LU[i,j]-sum([LU[i,k]*LU[k,j] for k in range(0,j)]))))
            else:  
                # Berechnung von U
                LU[i,j] = LU[i,j]-sum([LU[i,k]*LU[k,j] for k in range(0,i)])

        # Spalten-Pivotsuche:
        # Maximales Element in der aktuellen Zeile von U
        maxel = la.norm(LU[i,i:N],np.inf)

        valmax = abs(LU[i,i]) # LU[i,i] ist kritisches Element
        n = i # (Zaehl)index der Zeile mit dem groesstem Element

        if (valmax/maxel < 0.25):  # Probleme mit divide by 0?
            for k in range(i+1,N):
                tmp = abs(LU[i,k])
                if tmp > valmax:
                    n = k
                    valmax = tmp
            npivot = npivot + 1

            if not (i == n):
                #Vertausche LU[:][i] mit LU[:][n] und P[i][:] mit P[n][:]
                # In-place switching
                LU[:,i], LU[:,n] = LU[:,n], LU[:,i].copy()
                P[n,:], P[i,:] = P[i,:], P[n,:].copy()    

    return (LU,P)

### Lösung mit LUP

In [9]:
%reset -f array
A = np.array([
    [0,5,0,4],
    [-1,-6,0,-2],
    [9,3,-2,1],
    [-4,7,-1,3]])
y = np.array([
    [5],
    [-1],
    [2],
    [3]])
LU,P = LUPFaktorisierung(A)

EPSILON =  0.01 # Relative Toleranz
JMAX    =  10 # Max. Iterationen
N       =  LU.shape[0] # Dimension von L/U/P

j =  0  # Anzahl Iterationen
r =  -1*np.copy(y) # Residuum
x =  np.zeros([N,1]) # Lösungsvektor
dx =  np.zeros([N,1]) # Lösungsvektor
z =  np.zeros([N,1])  # Zwischenvektor für Vor-/Rückwärtseinsetzen
v =  np.zeros([N,1])  # Zwischenvektor für Permutation
e =  la.norm(r,2)/la.norm(y,2)  # Relativer Fehler

while (e > EPSILON and j <= JMAX):
    # Lz = y lösen nach z:
    z[0] = y[0]
    for i in range(1,N):
        z[i] = y[i]-sum([LU[i,k]*z[k] for k in range(0,i)])

    # Uv = z lösen nach v:
    v[N-1] = z[N-1]/LU[N-1,N-1]
    for i in reversed(range(0,N-1)):
        v[i] = (1/LU[i,i])*(z[i] - sum([LU[k,i]*v[i] for k in range(i+1,N)]))
    # x aktualisieren:
    x = (P.T).dot(v)
    # Zähler inkrementieren:
    j=j+1  
    # Residuum aktualisieren
    r = A.dot(x)-y
    # Relativen Fehler berechnen:
    e = la.norm(r,2)/la.norm(y,2)
    
    print("r: {}".format(r))
    print("e: {}".format(e))

print(LU.dot(P) - A)

r: [[  7.39495798]
 [ -3.69747899]
 [-40.75048229]
 [ 30.24660759]]
e: 8.233477366931409
r: [[   6.77511902]
 [ -27.82755951]
 [ 183.81455797]
 [ -70.73827163]]
e: 31.869915924051398
r: [[   6.90750823]
 [  97.01664588]
 [-941.10754473]
 [ 429.44422028]]
e: 166.37659570452573
r: [[    6.88713516]
 [ -527.95883158]
 [ 4683.880194  ]
 [-2070.58437152]]
e: 824.3854163327537
r: [[  6.89041457e+00]
 [  2.59703723e+03]
 [ -2.34411178e+04]
 [  1.04294202e+04]]
e: 4129.330552046782
r: [[  6.88989034e+00]
 [ -1.30279621e+04]
 [  1.17183882e+05]
 [ -5.20705805e+04]]
e: 20639.23036026411
r: [[  6.88997423e+00]
 [  6.50970378e+04]
 [ -5.85941118e+05]
 [  2.60429420e+05]]
e: 103203.57040774572
r: [[  6.88996081e+00]
 [ -3.25527962e+05]
 [  2.92968388e+06]
 [ -1.30207058e+06]]
e: 516010.43267182214
r: [[  6.88996296e+00]
 [  1.62759704e+06]
 [ -1.46484411e+07]
 [  6.51042942e+06]]
e: 2580059.5825736644
r: [[  6.88996261e+00]
 [ -8.13802796e+06]
 [  7.32421839e+07]
 [ -3.25520706e+07]]
e: 12900290.49

### Lösung mit Scipy LU

In [10]:
from scipy.linalg import lu,lu_factor,lu_solve
A = np.array([
    [0,5,0,4],
    [-1,-6,0,-2],
    [9,3,-2,1],
    [-4,7,-1,3]])
y = np.array([
    [5],
    [-1],
    [2],
    [3]])
print(A)
P,L,U = lu(A) # nutzt permutation matrix
print ("L:")
print (L)
print ("U:")
print (U)
print ("P:")
print (P)
lu = lu_factor(A)
lu_solve(lu,y)

[[ 0  5  0  4]
 [-1 -6  0 -2]
 [ 9  3 -2  1]
 [-4  7 -1  3]]
L:
[[ 1.          0.          0.          0.        ]
 [-0.44444444  1.          0.          0.        ]
 [-0.11111111 -0.68        1.          0.        ]
 [ 0.          0.6        -0.75221239  1.        ]]
U:
[[ 9.          3.         -2.          1.        ]
 [ 0.          8.33333333 -1.88888889  3.44444444]
 [ 0.          0.         -1.50666667  0.45333333]
 [ 0.          0.          0.          2.27433628]]
P:
[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]]


array([[ 0.01167315],
       [-0.43190661],
       [-0.70038911],
       [ 1.78988327]])